In [10]:
import math
import sys
from typing import List
from functools import lru_cache, reduce, cmp_to_key
from itertools import starmap, accumulate, chain, islice, product, tee, pairwise
from collections import Counter, deque
from heapq import heapify, heappushpop, heappop, heappush
from operator import add, neg
from bisect import bisect_right
import matplotlib.pyplot as plt

import arrays
import trees
from importlib import reload
reload(arrays)
reload(trees)
import arrays as ar
import trees as tr
import lists as ls
from lists import Node
print()

In [29]:
def queens_count(n: int) -> int:
    if n <= 0:
        return 0
    solutions = 0
    n4 = n * 4
    def rec(r: int, invalid: int):
        nonlocal solutions
        for c in range(n):
            place = (1 << c) + + (1 << (n + c + r)) + (1 << (n4 + c - r))
            if invalid & place:
                continue
            if r == 0:
                solutions += 1
                return
            rec(r-1, invalid | place)
    rec(n-1, 0)
    return solutions


In [32]:
list(map(queens_count, range(0, 14)))

[0, 1, 0, 0, 2, 10, 4, 40, 92, 352, 724, 2680, 14200, 73712]

In [14]:
import board as b
len(b.queens(7))

40